# 二値化    

目標:患者ごとに6つのROIsを100秒単位で分ける. 31×180このファイル.dat   
nameフォルダの中にID_6_180.datみたいな感じ

In [215]:
import matlab
import scipy 
from scipy import io
import pandas as pd
from pandas import DataFrame, Series
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import chainer.functions as F
import chainer.links as L
from chainer import Variable, optimizers, Chain, cuda
#import cupy as xp
import math
import datetime

import json
import sys
from sklearn import preprocessing

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [222]:
sys.getdefaultencoding()

'utf-8'

In [216]:
Data_Table = pd.read_csv('data_table.csv')
Data_Table

,PATH,ID,Ori_ID,Version,label,sex,Result_PATH
0,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0036v1,TD0036,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
1,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0216v1,CL0216,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
2,/work/project/MEG_ChildrenBiomarker/data/S1TD0...,S1TD0032v1,TD0032,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
3,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0044v1,CL0044,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
4,/work/project/MEG_ChildrenBiomarker/data/S1CL0...,S1CL0045v1,CL0045,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
5,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0008v1,CL0008,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
6,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0056v1,TD0056,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
7,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0138v1,CL0138,1.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
8,/work/project/MEG_ChildrenBiomarker/data/S2CL0...,S2CL0177v2,CL0177,2.0,1.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...
9,/work/project/MEG_ChildrenBiomarker/data/S2TD0...,S2TD0006v1,TD0006,1.0,0.0,1,/work/project/MEG_ChildrenBiomarker/SourceEsti...


In [353]:
def  function(PATH):
    list_dir = os.listdir(str(PATH)) 
    list_dir = [s for s in list_dir if ('data_block' in s)] 
    
    #matファイルの読み込み
    #data_block = io.loadmat(PATH + '/' + list_dir[0] ,squeeze_me = True)
    #results_MN_MEG_KERNEL = io.loadmat(PATH + '/' + list_dir[1], squeeze_me = True)
    data_block = [s for s in list_dir if 'data_block' in s]
    data_block = io.loadmat(PATH + '/' + data_block[0] ,squeeze_me = True)
    
    #変数の読み込み
    F = data_block["F"]
    
    #x<=0 は　-1, x>0 は1に二値化
    F[F>0] = 1
    F[F<=0] = -1

    ROIs_F = [F[i*6:(i+1)*6, :] for i in range(0,31)] #6*18000

    
    corr_PATH = np.where(Data_Table == PATH)
    tuple_list = [n for n in corr_PATH]
    a = tuple_list[0][0]
        
    #患者ごとのフォルダに保存する
    os.mkdir('/work/project/MEG_ChildrenBiomarker/binarized/' + Data_Table.ID[a])
    for i in range(0,31):
        np.savetxt('/work/project/MEG_ChildrenBiomarker/binarized/'+Data_Table.ID[a] +'/sample_%d.dat' %i, ROIs_F[i], fmt = '%.0f')
        
    result_path = [os.path.basename(r) for r in glob.glob('/work/project/MEG_ChildrenBiomarker/binarized/' + Data_Table.ID[a] + '/s*')]
    result_path = [result_path[r].rstrip('.dat') for r in range(0, 31)]
    result_path_num = [result_path[r][7:] for r in range(0, 31)]
    
    for j in range(0,31):
        ROIs_Fj = ROIs_F[int(result_path_num[j])]
        
        for k in range(0,100):
            times_F = [ROIs_Fj[:,180*k:180*(k+1)] for k in range(0,100)]
            np.savetxt('/work/project/MEG_ChildrenBiomarker/binarized/'+ Data_Table.ID[a] +'/'+result_path[j] +'_%d.dat' %k, times_F[k], fmt ='%.0f')
        
    return print('Data_Table.ID[%d]'%a)

In [354]:
for k in range(0, 100):
    times_F = [ROIs_Fj[:,180*k:180*(k+1)] for k in range(0, 100)]
times_F[0]

array([[    0,     1,     2, ...,   177,   178,   179],
       [18000, 18001, 18002, ..., 18177, 18178, 18179],
       [36000, 36001, 36002, ..., 36177, 36178, 36179],
       [54000, 54001, 54002, ..., 54177, 54178, 54179],
       [72000, 72001, 72002, ..., 72177, 72178, 72179],
       [90000, 90001, 90002, ..., 90177, 90178, 90179]])

In [356]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ[" CUDA_VISIBLE_DEVICES "] = "0"
function(Data_Table.PATH[136])

KeyError: 136